In [4]:
arrr = np.zeros(((5, 10)))
arrr.min(axis=0).shape

(10,)

In [149]:
import numpy as np
from numpy.linalg import eig
import itertools
import functools
def normalize_complex_arr(a: np.ndarray):
    a_oo = a - a.real.min() - 1j*a.imag.min() # origin offsetted
    return a_oo/np.abs(a_oo).max()
def norm_complex(arr: np.ndarray):
    if len(arr.shape) > 1:
        ret_out = np.zeros_like(arr)
        for i in range(arr.shape[0]):
            ret_out[i,:] = arr[i]/np.sqrt((np.abs(arr[i])**2).sum())  
        return ret_out
    else:
        return arr/np.sqrt((np.abs(arr)**2).sum())  

In [100]:
sx = np.array([[0, 1], [1, 0]])
sy = np.array([[0, 1j], [-1j, 0]])
sz = np.array([[1, 0], [0, -1]])
basis = np.stack([np.eye(2), sx, sy, sz])

n = 4 # Nb of qubits
J = 4**n
I = 6**n
d = R = 2**n # matrix dimension
A = 3**n
npa = np.array
b = npa(list(itertools.product(range(4), repeat=4)))
a = npa(list(itertools.product(range(1, 4), repeat=4)))
r = npa(list(itertools.product([0, 1], repeat=4)))

def projectors(idx_list, r_):
    evs = [eig(basis[i])[1] for i in idx_list]
    selected_evs = np.array([ev[:, r_[i]] for i, ev in enumerate(evs)])
    # print(selected_evs)
    ret = npa([np.outer(np.conj(ev), ev) for ev in selected_evs])
    # print(ret)
    return ret

# Pauli basis for n qubit
sig_b = npa([functools.reduce(np.kron, (basis[b[i,:], :, :])) for i in range(J)])

# Matrix P_{(r,a),b}
P_rab = np.zeros((I, J))
for j in range(J):
    tmp = np.zeros((R, A))
    for s in range(R):
        for l in range(A):
            val = np.prod(r[s, b[j] != 0])\
                * np.prod(a[l, b[j] != 0] == b[j, b[j]!=0])
            tmp[s,l] = val
    P_rab[:, j] = tmp.flatten(order="F")

# The projectors matrices
Pra = []
for j in range(A):
    for i in range(R):
        # print(a[j], r[i])
        # print(projectors(a[j], r[i]))
        # print()
        Pra.append(npa(functools.reduce(np.kron, projectors(a[j], r[i]))).flatten(order="F")[::-1])
    # break

In [165]:
# Pure state
# dens_ma = np.zeros((d,d))
# dens_ma[0,0] = 1
# Mixed state
u = norm_complex(np.random.multivariate_normal(np.zeros(d*2),np.eye(d*2)/100, size=(16)).view(np.complex128))
dens_ma = np.conj(u.T) @ u /d
print(dens_ma.shape)

Prob_ar = np.zeros((A, R))
if n==1:
    for i in range(A):
        for j in range(R):
            Prob_ar[i,j] = dens_ma.flatten(order="F") @ projectors(a[i], r[j])
else:
    for i in range(A):
        for j in range(R):
            Prob_ar[i,j] = np.diag(dens_ma @ npa(functools.reduce(np.kron, projectors(a[i], r[j])))).sum()
Prob_ar = np.real(Prob_ar)
# Nb of times we repeat the measurements
n_size = 2000
p_ra = np.zeros((R, A))
for i, x in enumerate(Prob_ar):
    H = np.random.choice(R, n_size, replace=True, p=x) #n_size elements
    out = []
    for s in range(R):
        out.append((H==s).sum()/n_size)
    p_ra[:, i] = out
# Transform matrix to vector form
p_ra1 = p_ra.flatten(order="F")
temp1 = p_ra1 @ P_rab
temp1 = temp1/d

# Calculate coefs rho_b
rho_b = [0] * J
for i in range(J):
    rho_b[i] = temp1[i]/(3**((b[i] == 0).sum()))

# Calculate density using inversion technique
rho_hat = np.zeros((d, d), dtype=np.complex128)
for s in range(J):
    rho_hat += rho_b[s] * sig_b[s]
u_hat = eig(rho_hat)[1]

# renormalize lambda_hat
lamb_til = eig(rho_hat)[0]
lamb_til[lamb_til < 0] = 0
lamb_hat = lamb_til/lamb_til.sum()
lamb_hat

(16, 16)


/tmp/ipykernel_17704/3572718805.py:17: ComplexWarning: Casting complex values to real discards the imaginary part
  Prob_ar[i,j] = np.diag(dens_ma @ npa(functools.reduce(np.kron, projectors(a[i], r[j])))).sum()


array([7.42373757e-01+1.94731385e-18j, 7.76147068e-02+5.77506632e-19j,
       6.50012919e-02+7.83379470e-18j, 5.29160344e-02-1.11454304e-17j,
       3.77209942e-02+1.45802472e-18j, 1.12809986e-02+2.77163744e-18j,
       0.00000000e+00+0.00000000e+00j, 7.41479856e-03+2.67516947e-18j,
       0.00000000e+00+0.00000000e+00j, 4.07788970e-03-2.32863038e-18j,
       1.53065230e-03-1.98204871e-18j, 0.00000000e+00+0.00000000e+00j,
       0.00000000e+00+0.00000000e+00j, 0.00000000e+00+0.00000000e+00j,
       0.00000000e+00+0.00000000e+00j, 6.88760397e-05-1.80733727e-18j])

## Main MH

In [ ]:
# Main part
rho = np.zeros((d, d))
Te = np.random.standard_exponential(d)
Id = np.eye(d)
U = u_hat
Lamb = Te/Te.sum()
ro = 1/2
S = (rho_hat + np.conj(rho_hat.T))/2
be = 1

gamm = n_size/2
entry = []
Iter = 500
burnin = 100

for t in range(Iter + burnin):
    for j in d:
        Te_can = Te.copy()
        Te_can[j] = Te[j] * np.exp(be * np.random.uniform(-0.5, 0.5, 1))
        L_can = Te_can/Te_can.sum()
        tem_can = (U @ np.diag(L_can) @ np.conj(U.T)).flatten(order="F")
        tem = (U @ np.diag(Lamb) @ np.conj(U.T)).flatten(order="F")
        ss = 



In [155]:
#np.ones((5, 10)).sum(axis=0)
# dens_ma.flatten() @ projectors(a[0], r[0])
p_ra

array([[0.0515, 0.045 , 0.0565, ..., 0.0615, 0.057 , 0.0495],
       [0.047 , 0.0575, 0.053 , ..., 0.0465, 0.038 , 0.0465],
       [0.061 , 0.054 , 0.067 , ..., 0.044 , 0.0435, 0.0505],
       ...,
       [0.036 , 0.0515, 0.0635, ..., 0.068 , 0.088 , 0.0875],
       [0.075 , 0.064 , 0.0685, ..., 0.09  , 0.068 , 0.0735],
       [0.068 , 0.0695, 0.071 , ..., 0.0705, 0.0825, 0.083 ]])

In [20]:
functools.reduce(np.kron, [np.eye(2)] * 4).shape

(16, 16)

In [134]:
np.random.multivariate_normal(np.zeros(d*2),np.eye(d*2)/100, 16).view(np.complex128)

array([[-0.06956268+0.02435846j, -0.14123075-0.06311271j,
         0.00222598+0.2609201j ,  0.05914259+0.08152464j,
        -0.03022365+0.05777163j,  0.02326001-0.08326039j,
        -0.0484067 +0.24582793j, -0.00301281-0.13517634j,
        -0.02032145+0.06912466j, -0.10548587-0.04543596j,
         0.07877333+0.08333259j, -0.10900354+0.03113042j,
        -0.06166528+0.16194991j,  0.00724261-0.03640795j,
        -0.06860541-0.11615742j, -0.02843429+0.03530594j],
       [-0.11057946-0.1204106j , -0.05896407+0.08861196j,
         0.11388097+0.13442166j, -0.06069431+0.05797215j,
         0.11024054+0.03092668j,  0.20188125+0.08169993j,
        -0.13928047-0.09284634j, -0.12224631+0.14017381j,
        -0.07238691-0.23412461j,  0.14743275-0.07562126j,
         0.08320665-0.04646794j, -0.06249008+0.02412488j,
         0.01166913-0.01180255j, -0.02507991-0.12516436j,
         0.02586024+0.11108448j, -0.14223691+0.11283102j],
       [ 0.11396121-0.09717666j, -0.13510539+0.10105309j,
         0.1

In [142]:
cpl_v = npa([-0.06956268+0.02435846j, -0.14123075-0.06311271j,
         0.00222598+0.2609201j ,  0.05914259+0.08152464j,
        -0.03022365+0.05777163j,  0.02326001-0.08326039j,
        -0.0484067 +0.24582793j, -0.00301281-0.13517634j,
        -0.02032145+0.06912466j, -0.10548587-0.04543596j,
         0.07877333+0.08333259j, -0.10900354+0.03113042j,
        -0.06166528+0.16194991j,  0.00724261-0.03640795j,
        -0.06860541-0.11615742j, -0.02843429+0.03530594j])

In [146]:
def norm_complex(arr: np.ndarray):
    ret_out = np.zeros_like(arr)
    if len(arr.shape) > 1:
        for i in range(arr.shape[0]):
            ret_out[i,:] = arr[i]/np.sqrt((np.abs(arr[i])**2).sum())  
    else:
        return arr/np.sqrt((np.abs(arr)**2).sum())       
norm_complex(cpl_v)

array([-0.12776887+0.04474027j, -0.2594048 -0.11592192j,
        0.00408856+0.47924354j,  0.10862982+0.14973993j,
       -0.05551312+0.10611172j,  0.04272269-0.15292806j,
       -0.08891074+0.45152308j, -0.00553376-0.24828439j,
       -0.03732531+0.12696434j, -0.19375059-0.08345425j,
        0.14468648+0.15306067j, -0.20021165+0.05717863j,
       -0.11326336+0.2974606j ,  0.01330282-0.0668721j ,
       -0.12601061-0.2133515j , -0.05222652+0.06484799j])